# My Second AI Agent

<div>
<center><img src="./images/shopping_agent.png" width="600"/></center>
</div>

Welcome to the second exercise! In this exercise, you will build a more complex AI agent: a **Shopping Assistant** for a clothing store.

### Objective:
The goal of this exercise is to create an AI agent that can:
1. Help customers find products by retrieving product information (e.g., "red t-shirt", "black shoes").
2. Calculate totals and discounts for selected items using a calculator tool (since agents aren’t always reliable at math)

### Why This Exercise?
This exercise will demonstrate how to:
1. **Coordinate Multiple Tools**: Learn how an agent can use more than one tool in a single session, switching between tools as needed to perform its tasks.
2. **Handle Complex Workflows**: Teach the agent to handle multi-step processes (e.g., retrieving product details, calculating prices, and applying discounts).

### What You’ll Build:
The Shopping Assistant AI agent will:
1. Take user questions like:
   - *“Can you show me the price of a blue jacket?”*
   - *“How much would two pairs of black shoes cost?”*
2. Retrieve product details using a product information tool.
3. Use a calculator tool to compute totals and apply discounts.
4. Provide meaningful responses to the user by combining outputs from multiple tools.

By the end of this exercise, you’ll have hands-on experience creating a multi-tool agent capable of performing more **complex operations** and reasoning about customer requests.

### Ready to Get Started?
Let’s begin by setting up the environment and preparing the tools for our smarter, multi-functional agent!

## 0. Preparing the Environment

In [ ]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from utils.stream import print_stream

load_dotenv(override=True)

## 1. Initializing the Large Language Model (LLM)

In [ ]:
MODEL_NAME = "openai:gpt-4o-mini-2024-07-18"
MODEL_TEMPERATURE = 0

llm = init_chat_model(model=MODEL_NAME, temperature=MODEL_TEMPERATURE)

## 2. Defining the Tools

To build the **Shopping Assistant AI agent**, we'll need two specific tools. While we’ll provide the tool for retrieving clothing data, you’ll get the opportunity to define the calculator tool (or use a prebuilt version).

#### Tools Overview:

1. **Clothing Data Retrieval Tool** (Provided)
   - This tool retrieves basic information about available clothing items, such as type, price, and discount.
   - The agent will use it to respond to queries like:
     - *"What’s the price of a red t-shirt?"*
     - *"Do you have black shoes?"*

2. **Calculator Tool** (To Be Defined)
   - The agent will use this tool to perform mathematical operations, such as:
     - Calculating totals for multiple items.
     - Applying discounts.
   - You can define your own tool (recomended) or use a prebuilt one.

Let’s start by defining the data and the tools.

In [ ]:
import enum

class ProductNames(enum.Enum):
    RedShirt = "red shirt"
    BlueJeans = "blue jeans"
    BlackShoes = "black shoes"
    WhiteSocks = "white socks"
    GrayHat = "gray hat"
    GreenScarf = "green scarf"
    YellowBelt = "yellow belt"
    PurpleGloves = "purple gloves"
    BrownJacket = "brown jacket"
    PinkSweater = "pink sweater"

PRODUCTS = [
    {
        "name": ProductNames.RedShirt.value,
        "price": 20.00,
        "discount": 0.10,
        "currency": "EUR",
        "description": "A stylish red shirt made from high-quality cotton.",
    },
    {
        "name": ProductNames.BlueJeans.value,
        "price": 40.00,
        "discount": 0.20,
        "currency": "EUR",
        "description": "A pair of comfortable blue jeans made from durable denim.",
    },
    {
        "name": ProductNames.BlackShoes.value,
        "price": 30.00,
        "discount": 0.15,
        "currency": "EUR",
        "description": "A pair of elegant black shoes made from genuine leather.",
    },
    {
        "name": ProductNames.WhiteSocks.value,
        "price": 5.00,
        "discount": 0.05,
        "currency": "EUR",
        "description": "A pack of soft white socks made from breathable cotton.",
    },
    {
        "name": ProductNames.GrayHat.value,
        "price": 10.00,
        "discount": 0.10,
        "currency": "EUR",
        "description": "A stylish gray hat made from lightweight fabric.",
    },
    {
        "name": ProductNames.GreenScarf.value,
        "price": 15.00,
        "discount": 0.12,
        "currency": "EUR",
        "description": "A warm green scarf made from wool.",
    },
    {
        "name": ProductNames.YellowBelt.value,
        "price": 8.00,
        "discount": 0.08,
        "currency": "EUR",
        "description": "A yellow belt made from synthetic leather.",
    },
    {
        "name": ProductNames.PurpleGloves.value,
        "price": 12.00,
        "discount": 0.10,
        "currency": "EUR",
        "description": "A pair of purple gloves made from soft fabric.",
    },
    {
        "name": ProductNames.BrownJacket.value,
        "price": 50.00,
        "discount": 0.25,
        "currency": "EUR",
        "description": "A brown jacket made from genuine leather.",
    },
    {
        "name": ProductNames.PinkSweater.value,
        "price": 25.00,
        "discount": 0.15,
        "currency": "EUR",
        "description": "A pink sweater made from high-quality cotton.",
    }
]

In [ ]:
from tools.math import prebuilt_calculator

@tool
def get_product_info(product_name: ProductNames) -> dict:
    """TODO: Use this space to describe this tool to the agent.
    Try to answer the following questions:
        - What does this tool do?
        - What is the input?
        - How should the input be formatted?
    You can also provide examples of how to use this tool to help the agent understand it better.
    """
    for product in PRODUCTS:
        if product["name"] == product_name.value:
            return product
    return None

@tool
def calculator(expression: str) -> float:
    """TODO: Use this space to describe this tool to the agent.
    Try to answer the following questions:
        - What does this tool do?
        - What is the input?
        - How should the input be formatted?
    You can also provide examples of how to use this tool to help the agent understand it better.
    """
    # TODO: Implement this function (reccomended) or uncomment the following line to use the prebuilt one.
    #   HINT: You can use the `ast` library (safe) or the `eval` function (unsage) to evaluate a python mathematical expression.
    # return prebuilt_calculator(expression)

Test both tools to ensure they work correctly before integrating them into the agent.

In [ ]:
get_product_info.invoke({"product_name": ProductNames.RedShirt})

In [ ]:
calculator.invoke({"expression": "2 + 2 * 10"})

## 3. Crafting the Agent System Prompt

Define a system prompt for your agent. The agent should:
1. Help customers browse a clothing shop by retrieving details about items (using the `get_clothing_info` tool).
2. Perform calculations, such as computing totals or applying discounts, with the `perform_calculation` tool.
3. Politely decline irrelevant or unsupported queries (e.g., unrelated to clothing or shopping).

In [ ]:
SHOPPING_AGENT_PROMPT = """"#TODO: Define your agent system prompt here."""

## 4. Building the AI Agent

Now that you have the tools and system prompt ready, it’s time to build the AI agent. The agent should have access to both tools and should also have short-term memory to keep track of the conversation context.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# Initialize MemorySaver
memory = MemorySaver()

graph = create_react_agent(
    model=llm,
    tools=[get_product_info, calculator],
    prompt=SHOPPING_AGENT_PROMPT,
    checkpointer=memory,
)

## 5. Testing the AI Agent

Now that the agent is fully built, it's time to test it! Interact with the agent using a variety of prompts to see how well it can assist users in shopping for clothing. For example, you can ask:
- *"Can you show me the price of a blue jacket?"*
- *"How much would two pairs of black shoes cost?"*
- *"What’s the total for a red t-shirt and a pair of jeans?"*

In [ ]:
from uuid import uuid4

config = {
    "configurable": {
        "thread_id": str(uuid4())
        }
}

In [ ]:
question = """How much does the red shirt cost?"""

inputs = {"messages": [("user", question)]}

print_stream(graph.stream(inputs, config, stream_mode="values"))

In [ ]:
question = """I also need a pair of blue jeans. How much is the total?"""

inputs = {"messages": [("user", question)]}

print_stream(graph.stream(inputs, config, stream_mode="values"))

In [ ]:
question = """And a pair of black shoes and a pack of white socks. How much is the total now?"""

inputs = {"messages": [("user", question)]}

print_stream(graph.stream(inputs, config, stream_mode="values"))

### Iterative Refinement:

Now that your agent is up and running, it’s time to test it with your own queries! This hands-on testing allows you to evaluate:
1. **How well the agent responds to expected queries** (e.g., asking the time in different locations).
2. **Whether it handles unexpected or ambiguous queries gracefully** (e.g., if no location is provided or irrelevant questions are asked).
3. **How effectively it uses the tools** to fulfill its purpose.

### Iterative Process:

Crafting the perfect agent is an **iterative process**. Use these trials to identify areas of improvement and adjust the agent as needed. Consider modifying the following aspects:
- **System Prompt**: Adjust the wording to clarify the agent’s purpose, add reasoning steps, or inject personality.
- **Tool Definition**: Rename tools, change their parameters, or improve their descriptions for clarity and better integration.
- **Model Settings**: Experiment with model temperature to balance determinism (`temperature=0`) and creativity (higher values).
- **Tool Limitations**: For example, add error handling if an invalid timezone is passed.

### Tips for Effective Refinement:
- If your agent isn’t responding as expected, start by reviewing the reasoning trace.
- Test with a wider variety of queries to account for edge cases.
- Iterate gradually, making small adjustments to prompts, tools, or configurations one step at a time.

Remember, building an AI agent is like sculpting—it evolves over time with refinement and experimentation.

In [ ]:
from uuid import uuid4

config = {
    "configurable": {
        "thread_id": str(uuid4())
        }
}

In [ ]:
question = """<YOUR QUESTION HERE>"""

inputs = {"messages": [("user", question)]}

print_stream(graph.stream(inputs, config, stream_mode="values"))

## 9. Conclusion

🎉 Congratulations on Completing Exercise 2 and the Hands-On Session! 🎉

Amazing work! You’ve reached the end of the session, and your progress and creativity are truly commendable.

### What You’ve Accomplished:
Over the course of these exercises, you’ve:
1. **Learned the Foundations of AI Agents**: From building simple agents to understanding reasoning and tool usage.
2. **Mastered Tool Coordination**: Built and optimized agents that leverage multiple tools to solve complex, real-world problems.
3. **Explored Creativity with Custom Agents**: Designed unique, custom AI agents tailored to tasks of your choice, integrating tools like web browsing, code execution, and API usage.
4. **Built Real-World AI Use Cases**: Gained hands-on experience creating conversational agents that could be applied to real-life scenarios.

You’ve not only implemented functional AI agents but also explored the limitless potential of **LangGraph** and built skills that are invaluable for designing **future-facing AI systems**.

### The Journey Doesn’t End Here 🚀

This hands-on session is just the beginning of your AI agent-building journey. Armed with the knowledge and tools you’ve gained, you’re now ready to:
1. **Build More Complex AI Systems**: Leverage your new skills to tackle advanced projects and integrate even more sophisticated workflows.
2. **Explore LangGraph Further**: Dive deeper into LangGraph’s features, tools, and debugging utilities to push your agents to the next level.
3. **Apply Skills in Real-World Scenarios**: Use what you've learned to create highly customized, task-specific AI agents for professional, creative, or personal projects.

### Thank You for Participating 💡

We are thrilled to have had you join us for this lab and witness your amazing progress. Seeing your creativity and enthusiasm reminds us why working with AI systems is so exciting—because the possibilities are endless.

If you’re eager to continue your learning:
- Visit the **official LangGraph documentation**: [LangGraph Docs](https://langchain-ai.github.io/langgraph/tutorials/introduction/)

### Keep Building and Innovating 🌟
Thank you, and we can’t wait to see the incredible AI agents you'll build in the future. Stay curious, keep experimenting, and continue pushing the boundaries of what’s possible!